In [2]:
## if true, will only run on one sample that is provided with code for grading purposes 
runonsample = True

## run on dataset is dependent on all data being available in the specified path
runondataset = False

In [3]:
if runondataset:
  from google.colab import drive
  drive.mount('/content/drive')
  rootPath = '/content/drive/My Drive/DL_CV_FinalProject/'
  #dataPath = '/content/data/'
  ourdata_path = rootPath + 'Datasets/our_fall_dataset/'
  urfd_path = rootPath + 'URFD_Videos/'

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [5]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
#from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import sys
import time
import pickle
import glob
import matplotlib.pyplot as plt
import tqdm
import pandas as pd 
import glob

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [6]:
## Check for GPU 
gpu_check = torch.cuda.is_available()
print(gpu_check)

if runondataset:
  ourDataset = sorted(glob.glob(ourdata_path + '*'))
  urfdDataset = sorted(glob.glob(urfd_path + '*'))

True


In [7]:
## Initialize class for predictor functions
class ReturnPredictions(object):
    def __init__(self,cfg):
        self.predictor = DefaultPredictor(cfg)
        self.metadata = MetadataCatalog.get(
                cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused")

    def _frame_from_video(self,video):
        print(video)
        while video.isOpened():
            success,frame = video.read()
            if success:
                yield frame
            else:
                break

    def run_on_video(self,video):
        frame_gen = self._frame_from_video(video)
        for frame in frame_gen:
            
            yield self.predictor(frame),frame

    def run_on_image(self,image):
      # Convert image from OpenCV BGR format to Matplotlib RGB format.
      yield self.predictor(image),image


## Load and setup model and predictor
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

predictor = ReturnPredictions(cfg)


model_final_a6e10b.pkl: 237MB [00:23, 10.1MB/s]                           


In [8]:
def getAvgVel(keypts_allFrames,kp_thresh = 0.09):
  numFrames,_,_ = keypts_allFrames.shape
   
  keypoint_dict = {
      "nose":[],
      "left_eye":[], "right_eye":[],
      "left_ear":[], "right_ear":[],
      "left_shoulder":[], "right_shoulder":[],
      "left_elbow":[], "right_elbow":[],
      "left_wrist":[], "right_wrist":[],
      "left_hip":[], "right_hip":[],
      "left_knee":[], "right_knee":[],
      "left_ankle":[], "right_ankle":[],
  }

  keypointNames_MasterList = list(keypoint_dict)

  keypoint_dict_diff = {
      "nose":[None],
      "left_eye":[None], "right_eye":[None],
      "left_ear":[None], "right_ear":[None],
      "left_shoulder":[None], "right_shoulder":[None],
      "left_elbow":[None], "right_elbow":[None],
      "left_wrist":[None], "right_wrist":[None],
      "left_hip":[None], "right_hip":[None],
      "left_knee":[None], "right_knee":[None],
      "left_ankle":[None], "right_ankle":[None],
  }

  for i,framePts in enumerate(keypts_allFrames):
    for j,kp in enumerate(framePts):
      kpName = keypointNames_MasterList[j]
      if kp[2] > kp_thresh:
        keypoint_dict[kpName].append(kp[:2])
      else:
        keypoint_dict[kpName].append(np.array([None]))

  for kpName in keypoint_dict:
    pts = keypoint_dict[kpName]
    for i in range(1,len(pts)):
      if (pts[i-1].any() != None) and (pts[i].any() != None):
        keypoint_dict_diff[kpName].append(getMag(pts[i]-pts[i-1]))
      else:
        keypoint_dict_diff[kpName].append(None)

  frameAvgDiff = [0]
  for i in range(1,numFrames):
    tmpDiffList_ = []
    for bodyPt in keypoint_dict_diff:
      tmpDiffList_.append(keypoint_dict_diff[bodyPt][i])

    tmpDiffList = np.array([v for v in tmpDiffList_ if v != None])

    if len(tmpDiffList) == 0:
      frameAvgDiff.append(None)
    else:
      diffMean = np.mean(tmpDiffList)
      diffSTD = np.std(tmpDiffList)
      distance_from_mean = abs(tmpDiffList - diffMean)
      max_deviations = 1
      not_outlier = tmpDiffList[distance_from_mean < max_deviations * diffSTD]
      if len(not_outlier) == 0:
        frameAvgDiff.append(None)
      else:
        frameAvgDiff.append(np.mean(not_outlier))

  return frameAvgDiff

def takeManualDeriv(signal,t):
  out_sig = []

  for i in range(1,len(signal)):
    if (signal[i]==None) or (signal[i-1]==None):
      out_sig.append(None)
    else:
      out_sig.append(signal[i]-signal[i-1])
  return out_sig,t[1:]

def scaleArea(orig_frame, targetArea=(512*288)):
  frameArea = orig_frame.shape[0]*orig_frame.shape[1]
  s = np.sqrt(targetArea/frameArea)
  dim = (int(orig_frame.shape[1]*s),int(orig_frame.shape[0]*s))
  resized = cv2.resize(orig_frame, dim, interpolation = cv2.INTER_AREA)
  return resized,s

def resizeKPB(keyPoints,bbox, scale):
  # rescale keypoints and bounding boxes to match new image dimensions 
  keyPoints[:,:-1] = keyPoints[:,:-1]*scale
  bbox = bbox*scale
  return keyPoints,bbox

def getMag(pt_diff):
  return np.sqrt(pt_diff[0]**2 + pt_diff[1]**2)

def movAvg(signal,time_v,winSize=5):
    avgSig = []
    for i in range(np.floor(winSize/2).astype(int),len(signal)-np.floor(winSize/2).astype(int)):
        start_id = i-np.floor(winSize/2).astype(int)
        end_id = i+np.ceil(winSize/2).astype(int)
        subSig = [v for v in signal[start_id:end_id] if v != None] 

        if len(subSig) != 0: 
          avgSig.append(np.mean(subSig))
        else:
          avgSig.append(None)
        
    t = time_v[np.floor(winSize/2).astype(int):np.floor(winSize/2).astype(int)+len(avgSig)]

    return t,avgSig

In [9]:
def run_detector_video(videoPath,vis=False):
  storeKeypoints = [] 
  storeFrames = [] # store grayscale frames

  # arrays of 0s to append when no person present 
  noPointsArray = np.zeros((17,3))
  personThresh = 0.8

  # create detectron points and bounding box generator 
  vid = cv2.VideoCapture(videoPath)
  output_generator = predictor.run_on_video(vid) 

  imgID = 0

  # iterate through frames 
  for output_frame in output_generator:

    if vis:
      print('---------------- Image Number: ',imgID)

    output = output_frame[0]
    frame = output_frame[1]

    # scale image down 
    g_frame,_ = scaleArea(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    storeFrames.append(g_frame)

    # extract values from output 
    keypoints_all = output['instances'].pred_keypoints.cpu().numpy()
    person_scores = output['instances'].scores.cpu().numpy()
    bound_box = output['instances'].pred_boxes.tensor.cpu().numpy()

    # extract indices of cases where person is detected 
    idx_scores_valid = np.argwhere(person_scores>personThresh)
    num_valid = len(idx_scores_valid)

    imgID += 1

    if len(idx_scores_valid) == 0:
      # if no person detected 

      if vis:
        print('Could not detect person!')
      storeKeypoints.append(noPointsArray)

    else:    
      if vis:
        print("All Scores:",person_scores)

      # resize RGB image for showing figure 
      imgCopy,scale_factor = scaleArea(cv2.cvtColor(frame.copy(),cv2.COLOR_BGR2RGB))

      # resize kps and bboxes 
      key,box = resizeKPB(keypoints_all[np.argmax(person_scores)],
                          bound_box[np.argmax(person_scores)], 
                          scale_factor)

      storeKeypoints.append(key)

      #draw bounding boxes and key points 
      if vis:
        print('---height,width',(abs(box[3] - box[1]),abs(box[2] - box[0])))
        cv2.rectangle(imgCopy, (box[0],box[1]), (box[2],box[3]), (255,0,0),2)
        for pt in key:
          #print('Confidence = ',pt[2])
          if pt[2] > 0.1:
            cv2.circle(imgCopy, (pt[0],pt[1]), 2, (0,255,0), -1)
        plt.imshow(imgCopy)
        plt.show()

  storeKeypoints_ = np.stack(storeKeypoints,axis=0)
  storeFrames_ = np.stack(storeFrames,axis=0)

  return storeKeypoints_

In [11]:
def getClass(kps,classThresh=3.5):
  numFrames = len(kps)
  avgVel = getAvgVel(kps,kp_thresh = 0.1)
  tvel,vel = movAvg(avgVel,np.arange(numFrames),winSize=5)
  acc,tacc = takeManualDeriv(vel,tvel)

  acc_nozero = [v for v in acc if v != None]
  print('acc max',np.max(acc_nozero))
  print('acc min',np.min(acc_nozero))

  f_pred = 0
  start_idx = None
  end_idx = None
  for idx,a in enumerate(acc):
    if a != None:
      # if a > the threshold and onset has not yet been detected
      if (a > classThresh) and (f_pred==0):
        f_pred=1
        start_idx = idx
      
      # if an onset has been predicted a goes below the -thresh then predict an ending 
      if (f_pred==1) and (a < -classThresh):
        end_idx = idx
        return f_pred,start_idx,end_idx
  
  if (start_idx == None) and (end_idx == None):
    return 0,0,0
  elif (start_idx != None) and (end_idx == None):
    end_idx = numFrames
    return f_pred, start_idx, end_idx

# RUN ON OUR DATASET AND SAVE RESULTS

In [12]:
if runondataset:
  outMaster_fname = rootPath + 'results/our_dataset_master.csv'
  saveData_fname = rootPath + 'results/our_dataset_dec17_HUMANPOSEONLY.csv'
  outMaster = pd.read_csv(outMaster_fname)

  for sampleName in ourDataset:
    sampleID = sampleName.split('/')[-1].split('.')[0]
    print('-----Running Sample:', sampleID)
    print('Fall Class = ', outMaster[outMaster.sequence_name==sampleID]['fall_label'] )
    keyPointsAllFrames= run_detector_video(sampleName)
    fallPred,fallStart,fallEnd = getClass(keyPointsAllFrames,classThresh=1.5)
    print('Fall Prediction = ', fallPred)
    outMaster.loc[outMaster.sequence_name==sampleID,'fall_pred'] = fallPred
    outMaster.loc[outMaster.sequence_name==sampleID,'fall_start_pred'] = fallStart
    outMaster.loc[outMaster.sequence_name==sampleID,'fall_end_pred'] = fallEnd
    #outMaster.to_csv(saveData_fname)

  outMaster.to_csv(saveData_fname)   

# RUN ON URFD DATASET AND SAVE RESULTS

In [13]:
if runondataset:
  outMaster_fname = rootPath + 'results/urfd_master.csv'
  saveData_fname = rootPath + 'results/urfd_output.csv'
  outMaster = pd.read_csv(outMaster_fname)

  for sampleName in urfdDataset:
    sampleID = sampleName.split('/')[-1][:-9]
    print('-----Running Sample:', sampleID)
    print('Fall Class = ', outMaster[outMaster.sequence_name==sampleID]['fall_label'] )
    keyPointsAllFrames= run_detector_video(sampleName)
    fallPred,fallStart,fallEnd = getClass(keyPointsAllFrames,classThresh=1.5)
    print('Fall Prediction = ', fallPred)
    outMaster.loc[outMaster.sequence_name==sampleID,'fall_pred'] = fallPred
    outMaster.loc[outMaster.sequence_name==sampleID,'fall_start_pred'] = fallStart
    outMaster.loc[outMaster.sequence_name==sampleID,'fall_end_pred'] = fallEnd
    #outMaster.to_csv(saveData_fname)

  outMaster.to_csv(saveData_fname) 

Please upload the following files into your colab runtime if running on colab (they are located within the submitted files):
*   our_dataset_master.csv
*   erica-11.mp4

If running locally, structure has been implemented.

In [ ]:
colab = False
local = True
ViewSetting = False ## Change view setting if you would like to see OF and Keypoints
if runonsample:
  if colab:
    annotationsFile = 'our_dataset_master.csv'
    sampleName = 'erica-11.mp4'
    outMaster = pd.read_csv(annotationsFile)
    sampleID = sampleName.split('/')[-1].split('.')[0]
    print('-----Running Sample:', sampleID)
    print('Fall Label = ', outMaster[outMaster.sequence_name==sampleID]['fall_label'].item() )
    print('Start Frame Ground Truth = ', outMaster[outMaster.sequence_name==sampleID]['fall_start_frame'].item() )
    print('End Frame Ground Truth = ', outMaster[outMaster.sequence_name==sampleID]['fall_end_frame'].item() )
    keyPointsAllFrames= run_detector_video(sampleName)
    fallPred,fallStart,fallEnd = getClass(keyPointsAllFrames,classThresh=1.5,vis=ViewSetting)
    print('Fall Predictions = ', fallPred)
    print('Start Frame Detected = ', fallStart)
    print('End Frame Detected = ',fallEnd)

  if local:
    annotationsFile = './our_dataset_master.csv'
    sampleName = './erica-11.mp4'
    outMaster = pd.read_csv(annotationsFile)
    sampleID = sampleName.split('/')[-1].split('.')[0]
    print('-----Running Sample:', sampleID)
    print('Fall Label = ', outMaster[outMaster.sequence_name==sampleID]['fall_label'].item() )
    print('Start Frame Ground Truth = ', outMaster[outMaster.sequence_name==sampleID]['fall_start_frame'].item() )
    print('End Frame Ground Truth = ', outMaster[outMaster.sequence_name==sampleID]['fall_end_frame'].item() )
    keyPointsAllFrames= run_detector_video(sampleName)
    fallPred,fallStart,fallEnd = getClass(keyPointsAllFrames,classThresh=1.5)
    print('Fall Predictions = ', fallPred)
    print('Start Frame Detected = ', fallStart)
    print('End Frame Detected = ',fallEnd)
